In [12]:
from Bio import Entrez

In [13]:
Entrez.email = 'chen1i6c04@gmail.com'

def einfo(db):
    with Entrez.einfo(db=db) as handle:
        record = Entrez.read(handle)
    for fileld in record['DbInfo']['FieldList']:
        print(fileld['Name'], fileld['Description'])

def egquery(term, db='sra'):
    with Entrez.egquery(term=term) as handle:
        record = Entrez.read(handle)
    for row in record["eGQueryResult"]:
        if row["DbName"]==db:
            count = int(row["Count"])
            return count

def esearch(term, db="sra", retmax=10000):
    with Entrez.esearch(db=db, term=term, retmax=retmax) as handle:
        record = Entrez.read(handle)
        return record['IdList']

def efetch(uid, db, rettype='xml', retmode="xml"):
    with Entrez.efetch(db=db, id=uid, rettype=rettype, retmode=retmode) as handle:
        record = handle.read()
    return record

In [20]:
db = "bioproject"

In [21]:
einfo(db=db)

ALL All terms from all searchable fields
UID Unique number assigned to publication
FILT Limits the records
ORGN Organism
PRJA Project Accession
TYPE Project Type
STPE Project Subtype
DATE Registration Date
TITL Title
CEN Submitter Organization(s)
ACCN Space delimited GenBank or RefSeq Replicon Accessions
RTYP Replicon Type
RNME Replicon Name
LTP Locus Tag Prefix
WORD Organism/Project Description
KWRD Keyword
PROP Project/Organism Properties
DTPE Project Data Type
GRNT Grant ID
FUND Funding Agency
PMID Pubmed ID
DOID DOI ID
PID Project ID
RELV Relevance
ANME Assembly Name
BPRJ BioProject ID or accession
TPRJ Top Bioproject ID
WGSA WGS Accessions
AACC Assembly Accession
ATNM Attribute Name
ATTR Attribute


In [49]:
import os
import pandas as pd

In [46]:
outbreak_info = pd.read_csv('/home/chen1i6c04/notebook/20200529_cgMLST_type/pasteur_outbreak.csv', index_col=0)

In [50]:
for bioproject_accession in outbreak_info['BioProject'].dropna().unique():
    term = f"{bioproject_accession}[PRJA]"
    idlist = esearch(term, db=db)
    fetch_result = efetch(idlist, db, 'xml', 'xml')
    export_file = os.path.join('/media/NGS/Data_Analysis/20200529_cgMLST_type', bioproject_accession + '.xml')
    with open(export_file, 'w') as f:
        f.write(fetch_result)